In [1]:
# ====== Importer les library ====== #
import pandas as pd
import numpy as np
import os

# ====== Déterminer les path ====== #
import os
cwd = os.getcwd()
parent = os.path.dirname(cwd)
data_path = parent + '\\data\\'

df =pd.read_hdf(cwd + '\\dataframe\\' + 'df_clean.h5')
df.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [14]:
posts = pd.read_csv(data_path + 'Posts.csv')

In [15]:
posts = posts[['title']]
posts.head()

,title
0,Une électrification à « la mesure des besoins ...
1,Les incohérences de l’ingérence
2,L’état d’urgence décrété à Sept-Îles en raison...
3,"Le Petit Champlain en chantier, des commerçant..."
4,Legault veut empêcher les antiavortement de fa...


In [25]:
post_test = posts.head(100).copy()

In [ ]:
relevant_secondtopics = ['actualités', 'politique', 'sports', 'divertissement', 'santé', 'éducation',
                            'hockey', 'musique', 'auto', 'affaires-locales', 'environnement', 'covid-19',
                            'science', 'voyages', 'mode', 'arts', 'alimentation', 'cinéma', 'technologie',
                            'culture', 'jeunesse', 'expositions', 'techno',
                            'théâtre', 'livres', 'voyages', 'recettes', 'arts-visuels', 'sexe', 'estrie',
                            'remparts', 'sciences', 'société', 'oceanic', 'vin', 'guerre',
                            'canadiens','celebrites']

relevant_maintopics = ['actualites', 'affaires', 'arts', 'monde', 'sports', 'vivre', 'chroniques', 'opinions',
                   'consommation', 'jeunesse', 'magazines', 'actualite', 'le-mag', 'science', 'cinema', 'auto',
                   'maison', 'covid-19', 'dossiers', 'programmation', 'quebec-billets', 'hockey-junior', 
                   'nos-recommandations', 'remparts', 'ma-region', 'events', 'protegez-vous', 'tokyo-2021', 
                   'lesasdelinfo']

In [35]:
relevant_secondtopics = ['actualités', 'politique', 'sports', 'divertissement', 'santé', 'éducation',
                         'hockey', 'musique', 'auto', 'affaires-locales', 'environnement', 'covid-19',
                         'science', 'voyages', 'mode', 'arts', 'alimentation', 'cinéma', 'technologie',
                         'culture', 'jeunesse', 'expositions', 'techno', 'théâtre', 'livres', 'voyages',
                         'recettes', 'arts-visuels', 'sexe', 'estrie', 'remparts', 'sciences', 'société',
                         'oceanic', 'vin', 'guerre', 'canadiens', 'celebrites']

# Transform list to dictionary
relevant_secondtopics_dict = {index: topic for index, topic in enumerate(relevant_secondtopics)}

In [44]:
# Préparer les inputs
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from transformers import BertTokenizer
import torch

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
tokens = post_test['title'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True, padding='max_length'))

# Pad the sequences
max_len = max([len(token) for token in tokens])
padded_tokens = pad_sequences(tokens, maxlen=max_len, dtype='long', value=0, truncating='post', padding='post')

# Create attention masks
attention_masks = [[float(i != 0.0) for i in token] for token in padded_tokens]



from transformers import CamembertTokenizer, CamembertForSequenceClassification, AdamW

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForSequenceClassification.from_pretrained("camembert-base")

optimizer = AdamW(model.parameters(), lr=2e-5)


# Convert labels to numerical format (if necessary)
labels = [relevant_secondtopics_dict[label] for label in post_test['label']]

# Convert data to PyTorch tensors
input_ids = torch.tensor(padded_tokens)
attention_masks = torch.tensor(attention_masks)
labels = torch.tensor(labels)

# Set the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Set the model to training mode
model.train()

# Forward pass
inputs = {
    'input_ids': input_ids,
    'attention_mask': attention_masks,
    'labels': labels
}
outputs = model(**inputs)
loss = outputs.loss

# Backward pass
loss.backward()
optimizer.step()
optimizer.zero_grad()

ImportError: 
CamembertTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [45]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, CamembertTokenizer, CamembertForSequenceClassification, AdamW
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to tokenize, pad, and convert data to PyTorch tensors
def preprocess_data(data, tokenizer, max_length):
    # Tokenize the text data
    tokens = data['title'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length'))
    
    # Pad the sequences
    padded_tokens = pad_sequences(tokens, maxlen=max_length, dtype='long', value=0, truncating='post', padding='post')
    
    # Create attention masks
    attention_masks = [[float(i != 0.0) for i in token] for token in padded_tokens]
    
    # Convert data to PyTorch tensors
    input_ids = torch.tensor(padded_tokens)
    attention_masks = torch.tensor(attention_masks)
    labels = torch.tensor([relevant_secondtopics_dict[label] for label in data['label']])
    
    return input_ids, attention_masks, labels

# Load the tokenizer and model
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForSequenceClassification.from_pretrained("camembert-base")
optimizer = AdamW(model.parameters(), lr=2e-5)

# Preprocess the data
input_ids, attention_masks, labels = preprocess_data(post_test, tokenizer, max_length=512)

# Set the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Set the model to training mode
model.train()

# Forward pass
inputs = {
    'input_ids': input_ids,
    'attention_mask': attention_masks,
    'labels': labels
}
outputs = model(**inputs)
loss = outputs.loss

# Backward pass
loss.backward()
optimizer.step()
optimizer.zero_grad()



ImportError: 
CamembertTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
